In [131]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sun Sep 11 2020

@author: kalyan
"""

## SOLOW GROWTH MODEL 
import sympy as sym
import numpy as np
import pandas as pd
import dash_table

import scipy.stats as stats
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Constant returns to scale SOLOW growth model

def get_output_schedule(alpha=0.5,tech_factor=1,tech_type='tfp', wage=1,rent=1, price=1):
    
    # Parameters defining the exogenous market conditions
    W = wage
    R = rent
    P = price 
    
    # Parameters defining the production tech
    α = alpha

    k = np.linspace(1,10,10,endpoint=True)
    l = np.linspace(1,10,10,endpoint=True)
    #k,l = np.meshgrid(k,l)
    prod_tech = tech_type
    
    if prod_tech == 'tfp':
        # Parameters defining the structure of an economy
        A = 1   # Solow Neutral -> Capital Enhancing Tech
        B = 1   # Harrod Neutral -> Labor Enhancing Tech
        T = tech_factor # Hicks Neutral -> Total Factor Productivity
    elif prod_tech == 'capital_enhance':
        # Parameters defining the structure of an economy
        A = tech_factor   # Solow Neutral -> Capital Enhancing Tech
        B = 1             # Harrod Neutral -> Labor Enhancing Tech
        T = 1             # Hicks Neutral -> Total Factor Productivity
    elif prod_tech == 'labor_enhance':
        # Parameters defining the structure of an economy
        A = 1             # Solow Neutral -> Capital Enhancing Tech
        B = tech_factor   # Harrod Neutral -> Labor Enhancing Tech
        T = 1             # Hicks Neutral -> Total Factor Productivity
    else :
        A=1
        B=1
        T=1
        
    K, L = sym.symbols('K L')

    # Output 
    y = T*((A*K)**(α))*((B*L)**(1-α)) 
    Y = sym.lambdify((K,L), y, modules = ["numpy"])
    Y = Y(k,l)
    #print(Y.shape)
    array= np.array([k,l,Y,P*Y,W*l,R*k,P*Y-W*l-R*k]).T
    df = pd.DataFrame(data=array, columns=['K','L','Physical_Output','Revenue','Labor_Cost','Rent_Cost','Profit'])
    return df

In [132]:
df  = get_output_schedule(0.5,1.25,'tfp',1,1,1)
print(df)

dt = dash_table.DataTable(
    id='table',
    columns=[{"name": i, "id": i, "deletable": True, "selectable": True} for i in df.columns],
    data=df.to_dict('records'),
    fixed_rows={'headers': True},
    #style_table ={'height':300},
    style_data_conditional = [
        {
        'if': {
                # since using .format, escape { with {{
                'filter_query': '{{Profit}} = {}'.format(df['Profit'].max())
            },
            'backgroundColor': '#85144b',
            'color': 'white'
        },
    ],
        editable=True,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable="multi",
        row_selectable="multi",
        row_deletable=True,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10,
)


      K     L  Physical_Output  Revenue  Labor_Cost  Rent_Cost  Profit
0   1.0   1.0             1.25     1.25         1.0        1.0   -0.75
1   2.0   2.0             2.50     2.50         2.0        2.0   -1.50
2   3.0   3.0             3.75     3.75         3.0        3.0   -2.25
3   4.0   4.0             5.00     5.00         4.0        4.0   -3.00
4   5.0   5.0             6.25     6.25         5.0        5.0   -3.75
5   6.0   6.0             7.50     7.50         6.0        6.0   -4.50
6   7.0   7.0             8.75     8.75         7.0        7.0   -5.25
7   8.0   8.0            10.00    10.00         8.0        8.0   -6.00
8   9.0   9.0            11.25    11.25         9.0        9.0   -6.75
9  10.0  10.0            12.50    12.50        10.0       10.0   -7.50
